In [2]:
import regex

In [3]:
from datasets import load_dataset

dataset = load_dataset("clarin-knext/fiqa-pl", name="corpus")

titles = dataset['corpus']['title']
not_empty_titles = titles = [title for title in titles if title != '']
print(f"There is {len(not_empty_titles)} not empty titles in the dataset")

/Users/mwardynski/Documents/ds/_semestr_9/przetwarzanie_jezyka_naturalnego/labs/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


There is 0 not empty titles in the dataset


In [4]:
corpus = dataset['corpus']

### Używane formaty podawania czasu
1. HH:MM
2. HH:MM:SS
Mogą istnieć dodatkowe elementy, jak specyfikacja strefy czasowej, lub części sekundy po przecinku

### Użyte formaty podawania daty
Za słownikiem WSJP, datą nazywamy "numer określonego dnia, miesiąca, roku wskazany zgodnie z kalendarzem" (https://wsjp.pl/haslo/podglad/25270/data), tożsamą definicję podaje PWN. Ponieważ w poleceniu jest podany przykład używający jedynie dzień i miesiąc, zostanie on również zaliczony przez wyrażenie regularne dla daty.

Wszelkie pozostałe formaty definiujące punkt w czasie, np.: miesiąc i rok, lub też sam rok, zostaną pominięte.

Narodowe Centrum Kultury w poniższym artykule https://nck.pl/projekty-kulturalne/projekty/ojczysty-dodaj-do-ulubionych/ciekawostki-jezykowe/czy-wiecie-jak-poprawnie-zapisujemy-date-,c,50017 oraz słownik PWN w tym artykule: https://sjp.pwn.pl/poradnia/haslo/zapis-daty;10340.html podają właściwe sposoby zapisywania daty, użyję ich jako bazę do definicji przypadków zapisu dat, które powinny zostać rozpoznane przez wyrażenie regularne:
- 18.05.2023, 3.6.2023, 4.5.06, ale też z użyciem myślnika 03-05-2024, 1-9-1995 - zapis wyłącznie cyframi arabskimi. Do sprawdzenia występowanie formatu amerykańskiego używającego / jako separatora.
- 15 września 2021, 4 września 2021, też spotykane z miesiącem w mianowniku: 8 styczeń 2012 (możliwe, że i w innych przypadkach, co trzeba sprawdzić). Tylko ten format będzie akceptowany bez podanego roku, akceptacja pozostałych formatów bez podanego roku skutkowałaby zbyt dużą liczbą niewłaściwie rozpoznanych fragmentów tekstu jako daty.
- 15 wrz 2021 - czyli to co wyżej, tylko z nazwą miesiąca zapisaną trzyliterowym skrótem
- 10 X 2018, miesiąc w postaci liczby porządkowej w zapisie rzymskim

Powyższe przypadki dopuszczają też wiele niekoniecznie właściwych formatów, jednak wielu autorów tekstu nie unika błędów przy podawaniu dat, stąd niektóre zasady zostały poluzowane, a niektóre przypadki zostaną sprawdzone przed właściwą definicją wyrażenia regularnego. Jednakże reguły zostają zacieśniane w momencie, gdy takie działanie znacznie redukuje liczbę false-positives.

Pierwszy podlinkowany artykuł wspomina też o formacie, w którym zapis daty jest kompletnie słowny (np. dwudziesty siódmy kwietnia dwa tysiące dwunastego roku), jednakże jest to tak rzadki przypadek, że nie będzie on brany pod uwagę.

In [ ]:
pattern = regex.compile(r"(?<!http\S+)(\d{1,4})\s*[\.\-](?P<month>\d{1,2})\s*[\.\-](\d{1,4}})?", flags=regex.IGNORECASE | regex.MULTILINE)

#### Sprawdzanie przypadków szczególnych

- format amerykański używający ukośnika (/)
- dzień + miesiąc w innym przypadku, niż dopełniacz lub mianownik.

In [65]:
am_date_pattern = regex.compile(r'(?<!http\S+)\d{1,2}\s*\/\s*\d{1,2}', flags=regex.IGNORECASE | regex.MULTILINE)

for entry in corpus:
    found = regex.findall(am_date_pattern, entry['text'])

    if found:
        print(entry['_id'])
        print(entry['text'])

687
Dlaczego bogaci, którzy kupują droższe domy, mieliby być subsydiowani przez tych, którzy płacą mniej, czy przez tych, którzy wynajmują? Osoba, która kupi dom o wartości 500 tys. w obszarze o wysokim podatku od nieruchomości, wygrywa dwukrotnie. Raz, ale mając więcej ulg podatkowych, będąc w stanie odliczyć więcej niż przeciętna osoba z klasy robotniczej, która kupuje dom za 250 tys. Ceny domów SF są kombinacją lub spekulacją inwestycji i słabą kontrolą rządu. Ceny SF są na poziomie Tokio, gdy gęstość zaludnienia wynosi 1/10. To jest porażka polityki samorządu, by budować więcej domów, a nie brak ulg podatkowych. Jeśli twoja społeczność chce płacić wyższe podatki za więcej lokalnych usług, moc dla ciebie i zachęcam do tego, ale nie powinieneś być subsydiowany przez tych, którzy tego nie robią. Imo, ludzie nie powinni być karani za nie posiadanie nieruchomości lub życie w stanie, w którym uważają, że podatek od sprzedaży jest lepszy niż dochód lub majątek.
1034
„To, co opisujesz, jes

Szukanie daty w formacie amerykańskim zawierającym jedynie dzień i miesiąc wprowadzi znacznie więcej fałszywych rezultatów, niż tych właściwych przedstawiających datę. Najczęstszymi false-positive są proporcje, nazwy własne lub adresy URL zawierające datę, ale i spację.

Skoro odnalezienie dat w formacie amerykańskim zawierających jedynie dzień i miesiąc jest tak nieopłacalne, to zobaczmy czy dodanie roku do wyrażenia coś zmieni.

In [68]:
am_date_pattern = regex.compile(r"(?<!http\S+)\d{1,4}\s*\/\s*\d{1,2}\/\s*\d{1,4}", flags=regex.IGNORECASE | regex.MULTILINE)

for entry in corpus:
    found = regex.findall(am_date_pattern, entry['text'])

    if found:
        print(entry['_id'])
        print(entry['text'])

6760
„***JamieJBartlett***: >&#91;2012/10/22&#93;&#91;17:30:33&#93; >[&#91;Przetłumacz&#93;](http:// translate.google.com/#auto|auto|Przy 34 798 słowach warunki PayPal%27 są o 4000 słów dłuższe niż Hamlet. %22Brevity to dusza dowcipu%22. %28Akt II, scena II%29 „google przetłumacz to tweet'): Przy 34 798 słowach warunki PayPal są o 4000 słów dłuższe niż Hamlet. „„Zwięzłość jest duszą dowcipu””. „Akt II, scena II” [„Ten komentarz został wysłane przez bota&#91;FAQ&#93;](http://www.reddit.com/r/tweet_poster/faq 'tweet_poster')[&#91;Czy pomyliłem się?&#93 ;](http://www.reddit.com/message/compose/?to=tweet_poster&subject=Error%20Report&message=[Ups!](http://reddit.com/r/business/comments/11wkuv\)%0d %0dProszę pozostawić temat i ten link bez zmian, ale zachęcamy do dodania opisu tutaj. 'zgłoś błąd')"
13187
Jeśli chodzi o punkt c, jeśli uważasz, że firmy muszą płacić powyżej poziomu świadczeń socjalnych, rozpocznij kampanię, aby stało się to prawem. Spółki są zobowiązane do działania w interes

Wynik powyższego kodu został przeglądnięty i z 95 odnalezionych dokumentów spełniających zadane wyrażenie regularne, jedynie następujących siedem jest faktycznymi datami w formacie amerykańskim: 307602, 320073, 338943, 359713, 395906, 397212, 456470. Zdecydowana większość wśród pozostałych to niewłaściwie sformatowane adresy URL, do których zakradła się spacja, a poza nimi wyrażenia określające proporcje, czy też nazwy własne.

Poniże sprawdzę inne przypadki miesiąca, niż mianownik i dopełniacz. Liczbę mnogą myślałem odrzucić bez sprawdzania, ale jednak dla pewności dopiszę do testowanych przypadków.

In [69]:
jan_pattern = "styczniowi|styczniem|styczniu|stycznie|styczniów|styczni[^a]|styczniom|styczniami|styczniach"
feb_pattern = "lutemu|lutym|lute[^g][^o]|lutych|lutymi"
mar_pattern = "marcowi|marcem|marcu|marce|marców|marcom|marcami|marcach"
apr_pattern = "kwietniowi|kwietniem|kwietniu|kwietnie|kwietni[^a]|kwietniów|kwietniom|kwietniami|kwietniach"
may_pattern = "majowi|majem|maju|maje|majów|majom|majami|majach"
jun_pattern = "czerwcowi|czerwcem|czerwcu|czerwce|czerwców|czerwcom|czerwcami|czerwcach"
jul_pattern = "lipcowi|lipcem|lipcu|lipce|lipców|lipcom|lipcami|lipcach"
aug_pattern = "sierpniowi|sierpniem|sierpniu|sierpnie|sierpni[^a]|sierpniów|sierpniom|sierpniami|sierpniach"
sep_pattern = "wrześniowi|wrześniem|wrześniu|wrześnie|wrześni[^a]|wrześniów|wrześniom|wrześniami|wrześniach"
oct_pattern = "październikowi|październikiem|październiku|październiki|październików|październikom|październikami|październikach"
nov_pattern = "listopadowi|listopadem|listopadzie|listopady|listopadów|listopadom|listopadami|listopadach"
dec_pattern = "grudniowi|grudniem|grudniu|grudnie|grudni[^a]|grudniów|grudniom|grudniami|grudniach"

pattern = regex.compile(rf"\d\s*({jan_pattern}|{feb_pattern}|{mar_pattern}|{apr_pattern}|{may_pattern}|{jun_pattern}|{jul_pattern}|{aug_pattern}|{sep_pattern}|{oct_pattern}|{nov_pattern}|{dec_pattern})", flags=regex.MULTILINE | regex.UNICODE)
for entry in corpus:
    found = regex.findall(pattern, entry['text'])

    if found:
        print(entry['_id'])
        print(entry['text'])


Jak widać, po podaniu numeru dnia, miesiące w innej odmianie przez przypadki niż mianownik lub dopełniacz nie występują, więc przy specyfikacji wyrażenia regularnego można je pominąć.

In [164]:
months_nom = ["styczeń", "luty", "marzec", "kwiecień", "maj", "czerwiec", "lipiec", "sierpień", "wrzesień", "październik", "listopad", "grudzień"]
months_gen = ["stycznia", "lutego", "marca", "kwietnia", "maja", "czerwca", "lipca", "sierpnia", "września", "października", "listopada", "grudnia"]
months_3 = ["sty", "lut", "mar", "kwi", "maj", "cze", "lip", "sie", "wrz", "paż", "lis", "gru"]
months_roman = ["", "II", "III", "IV", "V", "VI", "VII", "VIII", "IX", "X", "XI", "XII(?i)"]

months_nom_p = "|".join(months_nom)
months_gen_p = "|".join(months_gen)
months_3_p = "|".join(months_3)
months_roman_p = "(?-i)I"+"|".join(months_roman)+"(?i)I"

months_w_year = f"{months_3_p}|{months_roman_p}"
months_wo_year = f"{months_nom_p}|{months_gen_p}"
separator = r"(?:\s+|\s*[\.\-]\s*)"

date_pattern = regex.compile(rf"\s(\d{{1,4}}){separator}(?:(?:({months_wo_year})(?:{separator}(\d{{1,4}}))?)|(?:(?:(1?\d)|({months_w_year})){separator}(\d{{1,4}})))[\s\.\-,;:]", flags=regex.IGNORECASE | regex.MULTILINE)


dates = []

for entry in corpus:
    found = regex.findall(date_pattern, entry['text'])
    for date in found:
        dates.append(date)

len(dates)


1509

Powyższa list zapiera krotki gdzie poszczególne pozycje odpowiadają następującym wartościom:
1. Dzień, chyba że data jest w formacie YMD, wtedy rok
2. Pełna nazwa miesiąca w mianowniku lub dopełniaczu
3. Rok, jeśli został podany po miesiącu o pełnej nazwie. Jeśli data w formacie YMD, to wtedy dzień
4. Miesiąc w postaci cyfry arabskiej - może występować wyłącznie, gdy jest podany rok w grupie szóstej
5. Miesiąc w postaci trzyliterowej lub rzymskiej - może występować wyłącznie, gdy jest podany rok w grupie szóstej
6. Rok, gdy miesiąc znajduje się w grupie czwartej lub piątek. Dzień przy formacie daty YMD

Jak widać w powyższym opisie, specyfikacja miesiąca w postaci liczb arabskich, rzymskich lub trzyliterowego skrótu zostaje uznana wyłącznie, gdy data posiada też rok. W innym wypadku wyrażenie regularne zbierało zbyt dużo fałszywych wyników, gdyż zapisy np. przedziałów, nazw własnych itp. w dużej mierze też się kwalifikowały.

Generalnie sztuka tworzenia wyrażeń regularnych polega na odnalezieniu właściwego balansu pomiędzy wariantem restrykcyjnym i dość luźno dopasowującym. Wyrażenie regularne mają problem z uwzględnieniem kontekstu w efektywny sposób, więc zawsze należy się liczyć z pewną ilością zaakceptowanych próbek, które nie są prawidłowe.

In [ ]:
# def parse_date(date):


godzina HH.MM

jeśli pierwszy człon daty ma 4 cyfry, to ostatni powinien mieć 2

jeśli dwa człony, to nie cyfry, bo to bardzo dużo szumu wprowadza, jeden musi być miesiącem napisanym słownie.

Pusta linia przed datą, żeby ograniczyć false-positives nie związane z datami

Zmniejszając precyzyjność wyrażenia regularnego możemy znaleźć więcej dat zapisanych w złym formacie, ale znajdziemy też zdecydowanie więcej elementów nie będących datą, np. separator pomiędzy grupami w postaci "\s*[\.\-]?\s*" pozwala na dość luźne obchodzenie się z pustymi znakami i separatorami w dacie, ale dozwala też cyfry bez separatora.